Decide which metrics(attributes) to use to assess the best team performance in the regular season. These don’t have to be backed by hard-evidence, but try to use some simple graphs to show the relation between a metric and a team's win percentage. For instance, the R^2 (r-squared) metric should be able to accomplish this. 
Resource: https://www.teamrankings.com/nba/stats/ 

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


Metric #1: Effective Field Goal Percentage (eFG%)

In [ ]:
# Comparing efG to win% 
